##### this version adds more annotation to the summary file including amino acid change in order to make the lollipop plots

In [1]:
import re
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as ss
import matplotlib.pyplot as plt
from matplotlib import style, colors
from matplotlib import gridspec
import random
import string
import math
import re
import os
# matplotlib.style.use('ggplot')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
dft = pd.DataFrame(np.arange(8).reshape(2,4))
dft.head(2)
dft[4] = [[100, 200, 300], [5,6]]
dft
dft.explode(4)

,0,1,2,3
0,0,1,2,3
1,4,5,6,7


,0,1,2,3,4
0,0,1,2,3,"[100, 200, 300]"
1,4,5,6,7,"[5, 6]"


,0,1,2,3,4
0,0,1,2,3,100
0,0,1,2,3,200
0,0,1,2,3,300
1,4,5,6,7,5
1,4,5,6,7,6


#### what does multiple means?
* multi_hit means a gene hit by two or more mutations. 

* use the new strelka results

use all calls from strelka: this is prefered since this takes into consideration of all soamtic calls:snvs and indels, high/moderate/low/modifier impacts.

in most papers, they only report mutation in coding regions including splicing site, excluding non-coding regions, i will get these numbers as well. synonymous/non-synonymous mutation are only in coding regions.

a good post about match

https://stackoverflow.com/questions/24958358/use-regular-expression-in-python-to-find-two-strings-in-line

a table details the equivalence of ontology an classic effect description

http://snpeff.sourceforge.net/SnpEff_manual.html

# 1. parsing strelka vcf files

In [4]:
def parse_strelka_vcf(vcf):
    if os.stat(vcf).st_size == 0:
        print(f"WARNING: empty file>{vcf}")
        return pd.DataFrame()
    df = pd.read_csv(vcf, comment='#', sep='\t', header=None, low_memory=False)
    patient = vcf.split('/')[4]
    df = df[[0,1,3,4,7]]
    df.columns = ['chr', 'pos', 'ref', 'alt', 'effects']
    df = df[(df['effects'].str.contains("HIGH"))|(df['effects'].str.contains("MODERATE"))|(df['effects'].str.contains("LOW"))|(df['effects'].str.contains("MODIFIER"))]
#     print(df.head(2))
    if not df.empty:
        df['annotations'] = df['effects'].apply(lambda x: get_annotations(x))      
        df['patient'] = patient

    df = df.drop('effects', axis=1)
    return df

In [4]:
def wide_to_long(df):
    s = df.annotations.str.split('$', expand=True).stack()
    idx = s.index.get_level_values(0)
    print('length of s and idx', len(s), len(idx))
    dfn = df.loc[idx]
    dfn['effect'] = list(s)
    dfn.drop('annotations', inplace=True, axis=1)
    dfn['impact'], dfn['impact_type'], dfn['type'], dfn['AA_change'], dfn['gene'], dfn['transcript'] = dfn['effect'].apply(lambda x: x.split('@')).str
    dfn.drop('effect', axis=1, inplace=True)
    return dfn

In [3]:
# keep it easy for now pick HIGH, MODERATE and then LOW and the first transcript. could pull out all transcripts if needed
def get_annotations(line):
    print(line)
    effs = line.split('EFF=')[1].split(',')
    #     extract impact, impact_type, amino acid change, and gene
    effs = ['@'.join(list(np.array(re.split('\(|\|',ef))[[0,1,2,4,6,9]])) for ef in effs if ('HIGH' in  ef) or ('MODERATE' in ef) or ('LOW' in ef) or ('MODIFIER' in ef)]
    effs = list(set(effs))
    high = [ef for ef in effs if 'HIGH' in ef]
    moderate = [ef for ef in effs if 'MODERATE' in ef]
    low = [ef for ef in effs if 'LOW' in ef]
    modifier = [ef for ef in effs if 'MODIFIER' in ef]
    if high:
        anno = '$'.join(high)
    elif moderate:
        anno = '$'.join(moderate)
    elif low:
        anno = '$'.join(low)
    elif modifier:
        anno = '$'.join(modifier)
    else:
        print('ERROR!')
#     make sure the genes have the same name
#     genes = [ef.split('@')[2] for ef in effs]
#     print(anno)
    # anno= list of annotations
    return anno


In [5]:
# # example run
ft = '/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02020/hg19a/GSH/A37247_A56468/strelka/23223/bwa/results/passed.somatic.snvs.eff.dbSNP_v137.cosmic_v64.annotations.vcf'
ft = '/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02020/hg19a/GSH/A37247_A56468/strelka/59869/bwa/results/passed.somatic.snvs.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf'
# assert count_coding_mutations(ft) == 266
df = parse_strelka_vcf(ft)
dfn = wide_to_long(df)

df.head(2)
dfn.head(2)

NT=ref;QSS=21;QSS_NT=21;SGT=GG->AG;SOMATIC;TQSS=1;TQSS_NT=1;EFF=DOWNSTREAM(MODIFIER||4084||305|OR4F5|protein_coding|CODING|ENST00000335137||A),INTERGENIC(MODIFIER||||||||||A);ASP=true;RS=552439640;RSPOS=74092;SAO=0;SLO=true;SSR=0;VC=SNV;VP=0x050100000005000002000100;WGT=1;dbSNPBuildID=142
NT=ref;QSS=24;QSS_NT=24;SGT=CC->CT;SOMATIC;TQSS=2;TQSS_NT=2;EFF=INTERGENIC(MODIFIER||||||||||T);ASP=true;RS=865993915;RSPOS=77339;SAO=0;SLO=true;SSR=0;VC=SNV;VP=0x050100000005000002000100;WGT=1;dbSNPBuildID=147
NT=ref;QSS=19;QSS_NT=19;SGT=AA->AG;SOMATIC;TQSS=1;TQSS_NT=1;EFF=DOWNSTREAM(MODIFIER||3338|||RP5-857K21.3|lincRNA|NON_CODING|ENST00000357876||G),DOWNSTREAM(MODIFIER||600|||RP5-857K21.4|processed_transcript|NON_CODING|ENST00000440200||G),INTERGENIC(MODIFIER||||||||||G)
NT=ref;QSS=21;QSS_NT=21;SGT=CC->CT;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTERGENIC(MODIFIER||||||||||T);ASP=true;GENEINFO=LOC101928706:101928706;INT=true;RS=111933822;RSPOS=671598;SAO=0;SSR=0;VC=SNV;VP=0x050000080005000002000100;WGT=1;dbSN

NT=ref;QSS=58;QSS_NT=58;SGT=GG->AG;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTERGENIC(MODIFIER||||||||||A)
NT=ref;QSS=27;QSS_NT=27;SGT=GG->GT;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTERGENIC(MODIFIER||||||||||T);ASP=true;RS=757798717;RSPOS=159644294;SAO=0;SSR=0;VC=SNV;VP=0x050000000005000002000100;WGT=1;dbSNPBuildID=144
NT=ref;QSS=27;QSS_NT=27;SGT=GG->CG;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTERGENIC(MODIFIER||||||||||C)
NT=ref;QSS=48;QSS_NT=48;SGT=CC->CG;SOMATIC;TQSS=1;TQSS_NT=1;EFF=UPSTREAM(MODIFIER||2155|||ATP1A2|processed_transcript|CODING|ENST00000468587||G),DOWNSTREAM(MODIFIER||528|||ATP1A2|processed_transcript|CODING|ENST00000478587||G),INTRON(MODIFIER||||1020|ATP1A2|protein_coding|CODING|ENST00000361216|3|G),INTRON(MODIFIER|||||ATP1A2|processed_transcript|CODING|ENST00000472488|3|G),INTRON(MODIFIER||||1009|ATP1A2|protein_coding|CODING|ENST00000392233|3|G)
NT=ref;QSS=28;QSS_NT=28;SGT=CC->CT;SOMATIC;TQSS=1;TQSS_NT=1;EFF=DOWNSTREAM(MODIFIER||2352|||DCAF8|processed_transcript|CODING|ENST00000475733||T),DOWNS

NT=ref;QSS=26;QSS_NT=26;SGT=GG->CG;SOMATIC;TQSS=1;TQSS_NT=1;EFF=DOWNSTREAM(MODIFIER||4931|||KIF5C|retained_intron|CODING|ENST00000460377||C),INTRON(MODIFIER||||957|KIF5C|protein_coding|CODING|ENST00000435030|21|C),INTRON(MODIFIER||||862|KIF5C|protein_coding|CODING|ENST00000414838|19|C),INTRON(MODIFIER|||||KIF5C|processed_transcript|CODING|ENST00000464066|14|C),INTRON(MODIFIER||||725|KIF5C|protein_coding|CODING|ENST00000397413|14|C),INTRON(MODIFIER|||||KIF5C|retained_intron|CODING|ENST00000482151|2|C)
NT=ref;QSS=41;QSS_NT=41;SGT=CC->CG;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTRON(MODIFIER||||171|LYPD6|protein_coding|CODING|ENST00000334166|1|G),INTRON(MODIFIER||||171|LYPD6|protein_coding|CODING|ENST00000409381|1|G),INTRON(MODIFIER|||||LYPD6|nonsense_mediated_decay|CODING|ENST00000418762|1|G),INTRON(MODIFIER||||52|LYPD6|protein_coding|CODING|ENST00000414420|2|G|WARNING_TRANSCRIPT_INCOMPLETE),INTRON(MODIFIER|||||LYPD6|nonsense_mediated_decay|CODING|ENST00000392854|3|G)
NT=ref;QSS=40;QSS_NT=40;SGT=G

NT=ref;QSS=51;QSS_NT=51;SGT=AA->AG;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTRON(MODIFIER||||941|BBX|protein_coding|CODING|ENST00000325805|15|G),INTRON(MODIFIER||||911|BBX|protein_coding|CODING|ENST00000415149|14|G),INTRON(MODIFIER||||891|BBX|protein_coding|CODING|ENST00000402543|14|G),INTRON(MODIFIER||||628|BBX|protein_coding|CODING|ENST00000416476|14|G),INTRON(MODIFIER||||911|BBX|protein_coding|CODING|ENST00000406780|13|G),INTRON(MODIFIER|||||BBX|processed_transcript|CODING|ENST00000473542|4|G),INTRON(MODIFIER||||78|BBX|protein_coding|CODING|ENST00000443253|3|G|WARNING_TRANSCRIPT_NO_START_CODON)
NT=ref;QSS=29;QSS_NT=29;SGT=TT->GT;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTERGENIC(MODIFIER||||||||||G)
NT=ref;QSS=35;QSS_NT=35;SGT=GG->CG;SOMATIC;TQSS=2;TQSS_NT=2;EFF=INTRON(MODIFIER||||323|CD47|protein_coding|CODING|ENST00000361309|10|C),INTRON(MODIFIER|||||CD47|processed_transcript|CODING|ENST00000471694|1|C),INTRON(MODIFIER||||305|CD47|protein_coding|CODING|ENST00000355354|8|C),INTRON(MODIFIER||||85|CD47|p

NT=ref;QSS=20;QSS_NT=20;SGT=AA->AG;SOMATIC;TQSS=1;TQSS_NT=1;EFF=UPSTREAM(MODIFIER||4186|||RP11-55C6.1|processed_pseudogene|NON_CODING|ENST00000512950||G),INTERGENIC(MODIFIER||||||||||G)
NT=ref;QSS=21;QSS_NT=21;SGT=AA->AG;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTERGENIC(MODIFIER||||||||||G)
NT=ref;QSS=33;QSS_NT=33;SGT=GG->CG;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTERGENIC(MODIFIER||||||||||C)
NT=ref;QSS=42;QSS_NT=42;SGT=AA->AG;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTERGENIC(MODIFIER||||||||||G)
NT=ref;QSS=27;QSS_NT=27;SGT=TT->GT;SOMATIC;TQSS=2;TQSS_NT=2;EFF=INTERGENIC(MODIFIER||||||||||G)
NT=ref;QSS=21;QSS_NT=21;SGT=GG->GT;SOMATIC;TQSS=1;TQSS_NT=1;EFF=DOWNSTREAM(MODIFIER||1296|||PRKRIRP9|processed_pseudogene|NON_CODING|ENST00000503971||T),INTERGENIC(MODIFIER||||||||||T)
NT=ref;QSS=27;QSS_NT=27;SGT=CC->CT;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTERGENIC(MODIFIER||||||||||T)
NT=ref;QSS=29;QSS_NT=29;SGT=GG->CG;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTERGENIC(MODIFIER||||||||||C)
NT=ref;QSS=48;QSS_NT=48;SGT=GG->AG;SOMATIC;TQSS=1;TQS

...

NT=ref;QSS=24;QSS_NT=24;SGT=CC->CG;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTERGENIC(MODIFIER||||||||||G)
NT=ref;QSS=20;QSS_NT=20;SGT=GG->CG;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTRON(MODIFIER||||164|FAM18B2-CDRT4|protein_coding|CODING|ENST00000522212|5|C),INTRON(MODIFIER||||276|FAM18B2|protein_coding|CODING|ENST00000225576|5|C),INTRON(MODIFIER||||185|FAM18B2|protein_coding|CODING|ENST00000519970|5|C),INTRON(MODIFIER|||||FAM18B2-CDRT4|nonsense_mediated_decay|CODING|ENST00000557349|5|C),INTRON(MODIFIER|||||FAM18B2-CDRT4|nonsense_mediated_decay|CODING|ENST00000481756|4|C),INTRON(MODIFIER|||||FAM18B2-CDRT4|nonsense_mediated_decay|CODING|ENST00000518506|5|C),INTRON(MODIFIER|||||FAM18B2|nonsense_mediated_decay|CODING|ENST00000581273|1|C)
NT=ref;QSS=24;QSS_NT=24;SGT=TT->CT;SOMATIC;TQSS=1;TQSS_NT=1;EFF=UPSTREAM(MODIFIER||2994|||FAM18B2|processed_transcript|CODING|ENST00000583206||C),INTRON(MODIFIER||||164|FAM18B2-CDRT4|protein_coding|CODING|ENST00000522212|5|C),INTRON(MODIFIER||||276|FAM18B2|protein_coding|CO

NT=ref;QSS=26;QSS_NT=26;SGT=CC->CT;SOMATIC;TQSS=1;TQSS_NT=1;EFF=DOWNSTREAM(MODIFIER||4754||957|ZNF507|protein_coding|CODING|ENST00000544431||T),DOWNSTREAM(MODIFIER||2854||953|ZNF507|protein_coding|CODING|ENST00000311921||T),INTRON(MODIFIER|||||AC007773.2|lincRNA|NON_CODING|ENST00000592680|2|T)
NT=ref;QSS=20;QSS_NT=20;SGT=GG->AG;SOMATIC;TQSS=1;TQSS_NT=1;EFF=DOWNSTREAM(MODIFIER||2560|||Metazoa_SRP|misc_RNA|NON_CODING|ENST00000478944||A),INTRON(MODIFIER||||1050|ANKRD27|protein_coding|CODING|ENST00000306065|2|A),INTRON(MODIFIER||||389|ANKRD27|protein_coding|CODING|ENST00000587352|2|A),INTRON(MODIFIER||||29|ANKRD27|protein_coding|CODING|ENST00000586463|1|A|WARNING_TRANSCRIPT_INCOMPLETE),INTRON(MODIFIER|||||ANKRD27|nonsense_mediated_decay|CODING|ENST00000588700|2|A),INTRON(MODIFIER|||||ANKRD27|processed_transcript|CODING|ENST00000586693|2|A),INTRON(MODIFIER||||74|ANKRD27|protein_coding|CODING|ENST00000590519|2|A|WARNING_TRANSCRIPT_NO_STOP_CODON)
NT=ref;QSS=33;QSS_NT=33;SGT=GG->CG;SOMATIC;TQS

NT=ref;QSS=29;QSS_NT=29;SGT=CC->CT;SOMATIC;TQSS=2;TQSS_NT=2;EFF=INTERGENIC(MODIFIER||||||||||T)
NT=ref;QSS=23;QSS_NT=23;SGT=GG->AG;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTERGENIC(MODIFIER||||||||||A)
NT=ref;QSS=33;QSS_NT=33;SGT=GG->AG;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTERGENIC(MODIFIER||||||||||A)
NT=ref;QSS=26;QSS_NT=26;SGT=TT->AT;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTRON(MODIFIER||||516|ARHGEF9|protein_coding|CODING|ENST00000253401|2|A),INTRON(MODIFIER||||478|ARHGEF9|protein_coding|CODING|ENST00000374878|2|A),INTRON(MODIFIER||||414|ARHGEF9|protein_coding|CODING|ENST00000374870|1|A),INTRON(MODIFIER||||463|ARHGEF9|protein_coding|CODING|ENST00000437457|1|A),INTRON(MODIFIER|||||ARHGEF9|processed_transcript|CODING|ENST00000495564|3|A),INTRON(MODIFIER||||495|ARHGEF9|protein_coding|CODING|ENST00000374872|4|A)
NT=ref;QSS=19;QSS_NT=19;SGT=GG->CG;SOMATIC;TQSS=2;TQSS_NT=2;EFF=INTERGENIC(MODIFIER||||||||||C)
NT=ref;QSS=26;QSS_NT=26;SGT=AA->AT;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTERGENIC(MODIFIER||||||||||T)
NT=ref;QS

NT=ref;QSS=20;QSS_NT=20;SGT=CC->CT;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTERGENIC(MODIFIER||||||||||T)
NT=ref;QSS=24;QSS_NT=24;SGT=AA->AG;SOMATIC;TQSS=2;TQSS_NT=2;EFF=INTERGENIC(MODIFIER||||||||||G)
NT=ref;QSS=30;QSS_NT=30;SGT=TT->AT;SOMATIC;TQSS=2;TQSS_NT=2;EFF=INTERGENIC(MODIFIER||||||||||A)
NT=ref;QSS=16;QSS_NT=16;SGT=GG->GT;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTERGENIC(MODIFIER||||||||||T)
NT=ref;QSS=15;QSS_NT=15;SGT=GG->GT;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTERGENIC(MODIFIER||||||||||T)
NT=ref;QSS=32;QSS_NT=32;SGT=AA->AC;SOMATIC;TQSS=1;TQSS_NT=1;EFF=DOWNSTREAM(MODIFIER||3087|||Metazoa_SRP|misc_RNA|NON_CODING|ENST00000580064||C),INTERGENIC(MODIFIER||||||||||C)
NT=ref;QSS=33;QSS_NT=33;SGT=TT->AT;SOMATIC;TQSS=1;TQSS_NT=1;EFF=DOWNSTREAM(MODIFIER||3139|||Metazoa_SRP|misc_RNA|NON_CODING|ENST00000580064||A),INTERGENIC(MODIFIER||||||||||A)
NT=ref;QSS=17;QSS_NT=17;SGT=GG->AG;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTERGENIC(MODIFIER||||||||||A)
NT=ref;QSS=15;QSS_NT=15;SGT=CC->CT;SOMATIC;TQSS=1;TQSS_NT=1;EFF=INTERGEN

NameError: name 'wide_to_long' is not defined

# 3. Strelka vcf files

In [13]:
wkdir = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/validation/'
f = 'strelka_classic_vcf_files.txt'
vcfs = pd.read_csv(f'{wkdir}{f}', header=None)[0].tolist()
vcfs[:2]
snvvcfs = [i for i in vcfs if 'snvs' in i]
len(snvvcfs)
snvvcfs[:2]
indelvcfs = [i for i in vcfs if 'indel' in i]
len(indelvcfs)
df = pd.DataFrame({'snv':snvvcfs, 'indel': indelvcfs})
df['patient'] = df.snv.apply(lambda x: x.split('/')[4])
df['pat2'] = df.indel.apply(lambda x: x.split('/')[4])

assert (df.patient == df.pat2).all() == True
df = df.drop('pat2', axis=1)
df.columns = ['strelka_snv_vcf', 'strelka_indel_vcf', 'patient']
df.set_index('patient', inplace=True)
df.head(2)

['/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02025/hg19a/GSH/A93956_A93945/strelka/61000/bwa/results/passed.somatic.indels.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf',
 '/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02025/hg19a/GSH/A93956_A93945/strelka/61000/bwa/results/passed.somatic.snvs.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf']

87

['/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02025/hg19a/GSH/A93956_A93945/strelka/61000/bwa/results/passed.somatic.snvs.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf',
 '/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02044/hg19a/GSH/A93957_A93946/strelka/61001/bwa/results/passed.somatic.snvs.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf']

87

,strelka_snv_vcf,strelka_indel_vcf
patient,,
HTMCP-03-06-02025,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...
HTMCP-03-06-02044,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...


In [15]:
ndf = df[['strelka_snv_vcf', 'strelka_indel_vcf']].dropna()
ndf.head(2)
ndf.shape

,strelka_snv_vcf,strelka_indel_vcf
patient,,
HTMCP-03-06-02025,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...
HTMCP-03-06-02044,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...


(87, 2)

In [10]:
# for tmp tesing
impacts = ['high', 'moderate', 'low', 'modifier']

f = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/validation/validation_2nd_strelka_20191010.xls'
ndf = pd.read_excel(f, header=None)
ndf.columns = ['patient', 'tlib', 'nlib', 'strelka_snv_vcf', 'strelka_indel_vcf']
ndf.set_index('patient', inplace=True)
ndf.head(2)
ndf = ndf.head(3)

,tlib,nlib,strelka_snv_vcf,strelka_indel_vcf
patient,,,,
HTMCP-03-06-02025,B68907,B68872,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...
HTMCP-03-06-02044,B68919,B68884,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...


# 4. concatenate snvs and indels 

In [28]:
dfmg = pd.DataFrame()
for ix, row in ndf.iterrows():
    patient = ix
    snv_vcf = row['strelka_snv_vcf']
    indel_vcf = row['strelka_indel_vcf']
    
    snv_df = parse_strelka_vcf(snv_vcf)
    indel_df = parse_strelka_vcf(indel_vcf)
    mdf = pd.concat([snv_df, indel_df], sort=False)
    mdf = mdf.drop_duplicates(keep='first')

    if dfmg.empty:
        dfmg = mdf
    else:
        dfmg = pd.concat([dfmg, mdf], sort=False)


In [29]:
dfmg.head()
dfmg.shape

,chr,pos,ref,alt,annotations,patient
0,1,121484536,C,T,INTERGENIC@MODIFIER@@@@,HTMCP-03-06-02025-01A-01D
1,1,121484671,T,G,INTERGENIC@MODIFIER@@@@,HTMCP-03-06-02025-01A-01D
2,1,121484700,T,A,INTERGENIC@MODIFIER@@@@,HTMCP-03-06-02025-01A-01D
3,1,121485137,G,C,INTERGENIC@MODIFIER@@@@,HTMCP-03-06-02025-01A-01D
4,2,92315416,T,C,INTERGENIC@MODIFIER@@@@,HTMCP-03-06-02025-01A-01D


(60, 6)

In [30]:
dfmgn = dfmg.reset_index(drop=True)

In [31]:
dfn = wide_to_long(dfmgn)
# wide_to_long(dfmg.iloc[46:416,:].reset_index(drop=True))


length of s and idx 75 75


In [32]:
dfn.shape
dfn.head()

(75, 11)

,chr,pos,ref,alt,patient,impact,impact_type,type,AA_change,gene,transcript
0,1,121484536,C,T,HTMCP-03-06-02025-01A-01D,INTERGENIC,MODIFIER,,,,
1,1,121484671,T,G,HTMCP-03-06-02025-01A-01D,INTERGENIC,MODIFIER,,,,
2,1,121484700,T,A,HTMCP-03-06-02025-01A-01D,INTERGENIC,MODIFIER,,,,
3,1,121485137,G,C,HTMCP-03-06-02025-01A-01D,INTERGENIC,MODIFIER,,,,
4,2,92315416,T,C,HTMCP-03-06-02025-01A-01D,INTERGENIC,MODIFIER,,,,


In [33]:
of = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/validation/strelka_HML_summary_20190624.txt'
of = "/projects/trans_scratch/validations/workspace/szong/cervical2/nbs/old_out.txt"
dfn.to_csv(of, index=False)

### make supplementary table, gene by patient matrix to indicate if a gene is mutated in each patient

In [22]:

df = pd.read_csv(of, sep='\t')
df.head(2)
df.shape

,chr,pos,ref,alt,patient,impact,impact_type,type,AA_change,gene,transcript
0,1,914517,C,T,HTMCP-03-06-02025,SYNONYMOUS_CODING,LOW,SILENT,R537,C1orf170,ENST00000433179
1,1,914517,C,T,HTMCP-03-06-02025,SYNONYMOUS_CODING,LOW,SILENT,R517,C1orf170,ENST00000341290


(43384, 11)

In [23]:
df1 = pd.pivot_table(df[['gene', 'patient', 'type']], index='gene', columns='patient', values='type', aggfunc=lambda x: 'mutated' if not x.empty else 'WT')

In [24]:
df1.head(2)
df1['number_patients_mutated'] = df1.notnull().sum(axis=1)
df1['percent_patients_mutated'] = df1['number_patients_mutated']/87.0
df2 = df1[['number_patients_mutated', 'percent_patients_mutated']]

df2.head()
of = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/validation/somatic_mutation_frequency_summary_20190624.tsv'
df2.to_csv(of, sep='\t')

patient,HTMCP-03-06-02025,HTMCP-03-06-02044,HTMCP-03-06-02060,HTMCP-03-06-02061,HTMCP-03-06-02062,HTMCP-03-06-02066,HTMCP-03-06-02067,HTMCP-03-06-02072,HTMCP-03-06-02081,HTMCP-03-06-02082,...,HTMCP-03-06-02409,HTMCP-03-06-02410,HTMCP-03-06-02412,HTMCP-03-06-02416,HTMCP-03-06-02421,HTMCP-03-06-02423,HTMCP-03-06-02425,HTMCP-03-06-02426,HTMCP-03-06-02439,HTMCP-03-06-02446
gene,,,,,,,,,,,,,,,,,,,,,
A2M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAK1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mutated,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


patient,number_patients_mutated,percent_patients_mutated
gene,,
A2M,2,0.022989
AAK1,2,0.022989
AARS,2,0.022989
AATK,3,0.034483
ABCA1,6,0.068966


In [25]:
(df1 == 'WT').any().any()

/projects/da_workspace/software/miniconda/envs/dlpy3/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


False

In [26]:
df1 = df1.fillna('wt')

In [27]:
of = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/validation/somatic_mutation_gene_by_patient_summary_20190624.tsv'
df1.to_csv(of, sep='\t')

In [31]:
df1.head(2)
df1.loc['PIK3CA', :].dropna()[:2]

patient,HTMCP-03-06-02025,HTMCP-03-06-02044,HTMCP-03-06-02060,HTMCP-03-06-02061,HTMCP-03-06-02062,HTMCP-03-06-02066,HTMCP-03-06-02067,HTMCP-03-06-02072,HTMCP-03-06-02081,HTMCP-03-06-02082,...,HTMCP-03-06-02412,HTMCP-03-06-02416,HTMCP-03-06-02421,HTMCP-03-06-02423,HTMCP-03-06-02425,HTMCP-03-06-02426,HTMCP-03-06-02439,HTMCP-03-06-02446,number_patients_mutated,percent_patients_mutated
gene,,,,,,,,,,,,,,,,,,,,,
A2M,wt,wt,wt,wt,wt,wt,wt,wt,wt,wt,...,wt,wt,wt,wt,wt,wt,wt,wt,2,0.022989
AAK1,wt,wt,wt,wt,wt,wt,wt,wt,wt,mutated,...,wt,wt,wt,wt,wt,wt,wt,wt,2,0.022989


patient
HTMCP-03-06-02025         wt
HTMCP-03-06-02044    mutated
Name: PIK3CA, dtype: object

# how many genes we targeted:

### genes in design target files:

In [5]:
f = '/home/sequence/Sequencing_Validation/SVA02/SeqVal125/Bioinformatic_Analysis/Agilent_SureDesign/DesignAttempt02_KT.20180503/SeqVal125_final_concat_Targets.sorted.bed'
df = pd.read_csv(f, sep='\t', skiprows=2, header=None)
df.head()

,0,1,2,3
0,chr1,8799,12199,chr1:8800-12199
1,chr1,10088,10578,chr1:10089-10578
2,chr1,14999,17999,chr1:15000-17999
3,chr1,16131,16352,chr1:16132-16352
4,chr1,534399,537999,chr1:534400-537999


In [9]:
df = df.astype(str)

In [13]:
g1 = df[~df[3].str.startswith('chr')][3].unique()

In [14]:
len(g1)

4566

### genes we tried to target, we also targeted some non coding regions

In [17]:
f1 = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/validation/HIV_cervical_validation_target_candidates_20190309_added.xls'
df1 = pd.read_excel(f1, sheet_name='proposed_genes')
df1.head()
g2 = df1.gene.unique()
len(g2)

,gene,number_HTMCP_patients_with_mutation,reason_picked_for_capture,suggest_exclude,average_RPKM
0,MUC4,87,mutated_in_3_or_more_HTMCP_patient,YES,19.873095
1,TTN,49,mutated_in_3_or_more_HTMCP_patient,YES,0.010591
2,AHNAK2,43,mutated_in_3_or_more_HTMCP_patient,YES,20.152050
3,PIK3CA,41,tcga_and_landscape_significantly_mutated,NaN,4.438437
4,AC007731.1,33,mutated_in_3_or_more_HTMCP_patient,YES,0.000377


2840

In [20]:
gd = set(g2) -set(g1)
len(gd)

105

In [28]:
gd.head()

AttributeError: 'set' object has no attribute 'head'

In [32]:
'RP11-545J16.1' in g1
'RP11-545J16.1' in g2
'PSPC1' in g1
'PSPC1' in g2

False

True

True

True